# CONSULTAS

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db"  # nombre del servicio en docker-compose
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

# Leer una tabla completa
df = pd.read_sql("SELECT * FROM grado;", engine)
df.head()

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,"Conocer y apreciar la diversidad cultural, exp...","Graduados en diversidad cultural, patrimonio e..."
1,2,Antropología Social y Cultural,15,Conocer y apreciar la diversidad de sociedades...,"Graduados en diversidad cultural, patrimonio e..."
2,3,Bellas Artes,2,El Grado en Bellas Artes forma artistas plásti...,"Titulados en arte pueden ser pintores, esculto..."
3,4,Conservación y Restauración del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,ConservadorRestaurador en instituciones públic...
4,5,Artes Escénicas,2,El Grado en Artes Escénicas capacita a jóvenes...,Profesionales con proyección multidisciplinar ...


In [2]:
df.fillna({
        'nombre': 'Unknown', 
        'descripcion':'Este grado no contiene descripción',
        'salidas':'Este grado no contiene salidas'
        },
    inplace=True)
df

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,"Conocer y apreciar la diversidad cultural, exp...","Graduados en diversidad cultural, patrimonio e..."
1,2,Antropología Social y Cultural,15,Conocer y apreciar la diversidad de sociedades...,"Graduados en diversidad cultural, patrimonio e..."
2,3,Bellas Artes,2,El Grado en Bellas Artes forma artistas plásti...,"Titulados en arte pueden ser pintores, esculto..."
3,4,Conservación y Restauración del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,ConservadorRestaurador en instituciones públic...
4,5,Artes Escénicas,2,El Grado en Artes Escénicas capacita a jóvenes...,Profesionales con proyección multidisciplinar ...
...,...,...,...,...,...
224,225,Ciencia de Datos e Inteligencia Artificial,75,El grado forma profesionales versátiles con só...,"Capacitados para gestionar Big Data, tomar dec..."
225,226,Inteligencia Artificial,75,Este Grado en Inteligencia Artificial proporci...,"Especialistas en tecnología, incluyendo seguri..."
226,227,Computación e Inteligencia Artificial,75,Este Grado proporciona una base sólida en info...,"Especialistas en tecnología digital, incluyend..."
227,228,Ingeniería Robótica Sotware,75,Formar profesionales que diseñen algoritmos y ...,La robótica se utiliza en industria y servicio...


In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': '376db2e76ce4', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ladopV-BSW2lTXgQVwYb5Q', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [35]:
# indice semántico

index_mapping = {
    "properties": {
        "id": { "type": "integer" },        
        "nombre": { "type": "text" },     
        "id_area": { "type": "integer" },
        "descripcion": { "type": "text" },
        "descripcion_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionality of the embeddings
            "index": True,
            "similarity": "cosine"
        },
        "salidas": { "type": "text" },
        "salidas_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionality of the embeddings
            "index": True,
            "similarity": "cosine"
        }
    }
}

index_name = "informacion_grados"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(
    index=index_name,
    mappings=index_mapping
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'informacion_grados'})

In [36]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for descripciones
embeddings_descripcion = model.encode(df["descripcion"].values.tolist())

# Generate embeddings (vector representations) for salidas
embeddings_salidas = model.encode(df["salidas"].values.tolist())

In [37]:
import json
from elasticsearch.helpers import bulk

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings_descripcion[i + idx].tolist()
            doc["salidas_vector"]=embeddings_salidas[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, index_name)

Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 29 Errors: []


### Queries sintácticas

In [38]:
import json

res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "term": {
            "nombre": "industrial"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 190, Nombre Título: Ingeniería en Organización Industrial, Description: Un Ingeniero de Organización analiza y resuelve problemas para optimizar procesos empresariales, identificando prioridades estratégicas, recursos y beneficios. Su formación dual le permite tener una visión global de la empresa., Score: 4.1358886

Document ID: 192, Nombre Título: Ingeniería en Electrónica y Automática Industrial, Description: La titulación se centra en la tecnología punta y la creación de ingenieros mecánicos que mejoren producción y trabajo humano. Se aplican conocimientos a sistemas para mejorar procesos industriales, pero también se utilizan en medicina, agricultura, distribución de mercancías y energía., Score: 3.4236507

Document ID: 189, Nombre Título: Ingeniería en Diseño Industrial y Desarrollo de Producto, Description: La carrera de Diseño Industrial forma profesionales que diseñan productos innovadores, competitivos y eficientes desde el punto de vista funciona

In [39]:
res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "informática"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 214, Nombre Título: Ingeniería Informática, Description: La carrera de Ingeniería Informática forma profesionales que crean programas informáticos ajustados a las necesidades sociales y empresariales. Los ingenieros informáticos utilizan lenguajes de programación para analizar problemas cotidianos y desarrollar soluciones lógicas. La titulación se centra en la gestión y explotación de sistemas informáticos, incluyendo el diseño y desarrollo de software orientado al usuario final., Score: 5.2223105

Document ID: 215, Nombre Título: Ingeniería Informática Biomédica, Description: El grado en Ingeniería Informática Biomédica se enfoca en la aplicación de tecnología, informática y programación en salud y biomedicina., Score: 4.616037

Document ID: 220, Nombre Título: Matemáticas e Informática, Description: Este grado combina Matemáticas e Informática, enfatizando la interrelación entre ambas. Forma graduados con conocimientos generales en ambos campos para acced

### Queries léxcias

In [42]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "match": {
            "descripcion": "computación"
        }
    }
)


print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 203, Nombre Título: Matemática Computacional, Description: El Grado en Matemática Aplicada y Computación forma profesionales que aplican matemáticas en áreas como la ciencia, ingeniería y economía. Los estudiantes aprenden algoritmos, métodos numéricos y programación para resolver problemas reales., Score: 4.7130547

Document ID: 225, Nombre Título: Ciencia de Datos e Inteligencia Artificial, Description: El grado forma profesionales versátiles con sólida base en Matemáticas, Estadística y Computación. Los egresados pueden aplicar habilidades en tecnologías de la información, ciencia de datos e inteligencia artificial para diseñar estrategias de gestión de datos y sistemas de información. También deben considerar aspectos éticos, legales y sostenibles en el uso de técnicas de IA y datos., Score: 3.658999



In [ ]:
import 